In [1]:
from pypdb import Query
from tqdm.notebook import tqdm

In [2]:
number_to_sequence = {}

In [3]:
with open("./data/TriZOD_test_set.fasta.txt", "r") as f:
    # number in first lines, sequence in second
    while True:
        number = f.readline()
        sequence = f.readline()
        if not sequence:
            break
        number_to_sequence[number.strip()[1:]] = sequence.strip()

In [4]:
def get_exact_pdb_match(seq):

    q = Query(seq, 
            query_type="sequence", 
            return_type="polymer_entity",
            )
    search_res = q.search()
    
    if search_res is None:
        return None

    only_1_match = [x for x in search_res["result_set"]
                    if x['services'][0]["nodes"][0]['match_context'][0]['sequence_identity'] == 1.0 and
                    x['services'][0]["nodes"][0]['match_context'][0]['alignment_length'] == len(seq)]
    
    if len(only_1_match) == 1:
        return only_1_match[0]
    else:
        return None

In [5]:
our_index_to_pdb = {}

for key in tqdm(number_to_sequence):
    pdb_match = get_exact_pdb_match(number_to_sequence[key])
    if pdb_match:
        our_index_to_pdb[key] = pdb_match

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))


  0%|          | 0/348 [00:00<?, ?it/s]

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/pypdb/util/http_requests.py:65: UserWarning: Too many failures on requests. Exiting...
  warnings.warn("Too many failures on requests. Exiting...")
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/pypdb/pypdb.py:303: UserWarning: Retrieval failed, returning None
  warnings.warn("Retrieval failed, returning None")


In [6]:
[x['identifier'].split('_')[0] for x in our_index_to_pdb.values()]

['2RML',
 '2RPB',
 '2RPA',
 '2RRE',
 '1V6G',
 '2RRN',
 '2OSG',
 '2JOD',
 '2HGK',
 '2JSD',
 '2JX5',
 '2KGF',
 '2JZ4',
 '2JZC',
 '2JZV',
 '2K6V',
 '2KCX',
 '2KEP',
 '2KG7',
 '2KGT',
 '2KK6',
 '2LEK',
 '2KMG',
 '2KNE',
 '2KNI',
 '2L3U',
 '2KQM',
 '2L3O',
 '6N2M',
 '6NL3',
 '6NNB',
 '6O0I',
 '6O1Q',
 '6OBK',
 '6PRQ',
 '6VGB',
 '6X1K',
 '7KBQ',
 '7KIX',
 '7R6P',
 '7UWY',
 '5LBJ',
 '5LFI',
 '5MSL',
 '5MWW',
 '5N7Y',
 '5NAM',
 '5NB9',
 '5NBB',
 '5OEO',
 '6G5S',
 '6HZ2',
 '6R5G',
 '6TAZ',
 '6TOB',
 '6XTT',
 '7ASY',
 '7NE8',
 '7OMK',
 '7QBY',
 '7QIL',
 '7QJF',
 '7YWQ',
 '8AR2',
 '5X5S',
 '5X9X',
 '6LQZ',
 '7BPL',
 '7BPN',
 '7BPP',
 '7BQB',
 '7BQD',
 '7BQE',
 '7BQN',
 '7BQR',
 '7D2O',
 '7VKV',
 '6T33',
 '7R68',
 '8E6Y',
 '2JXY',
 '2BV2',
 '5ZSW',
 '7NSC',
 '5VXB',
 '7BY7',
 '6L7Z',
 '6YSE',
 '5KES',
 '5KPH',
 '6BF2',
 '5SXY',
 '5T3Y',
 '5T82',
 '5UOI',
 '5UTG',
 '5VNT',
 '5WOX',
 '5WOY',
 '6C8U',
 '6CGH',
 '6DMP',
 '6DSL',
 '6E5N',
 '4BXL',
 '4BXL',
 '4BYA',
 '2MHY',
 '4AIX',
 '2MIZ',
 '2MJM',
 

In [25]:
# from Bio.PDB import PDBList


# pdbl = PDBList()

# pdbl.download_pdb_files([x['identifier'].split('_')[0] for x in our_index_to_pdb.values()], pdir="./pdb_files", file_format="pdb")

Structure exists: './pdb_files/pdb4bxl.ent' 


In [12]:
from Bio.PDB import PDBParser, PDBIO

# Load the PDB file
pdb_parser = PDBParser()
structure = pdb_parser.get_structure(file="pdb_files/pdb1v6g.ent")

# Get the first model
first_model = next(structure.get_models())

# Write the first model to a new PDB file
# io = PDBIO()
# io.set_structure(first_model)
# io.save("first_model_only.pdb")

print("First model extracted and saved as 'first_model_only.pdb'")

TypeError: PDBParser.get_structure() missing 1 required positional argument: 'id'

In [19]:
our_index_to_pdb_files = {}

for k, v in our_index_to_pdb.items():
    prot, model_id = v['identifier'].split('_')
    #print(model_id)
    pdb_parser = PDBParser()
    structure = pdb_parser.get_structure(prot, f"pdb_files/pdb{str.lower(prot)}.ent")
    model = list(structure.get_models())[0]
    our_index_to_pdb_files[k] = model

/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3115.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5880.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8645.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11410.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 14175.
  warnings.warn(
/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packag

In [22]:
model = our_index_to_pdb_files["11011_1_1_1"]

In [40]:
residues_str = [x for x in model.get_residues()]

In [42]:
residues_str[0]

<Residue MET het=  resseq=1 icode= >

In [43]:
residues_str = [x.get_resname()[0] for x in model.get_residues() if x.get_resname() != "HOH"]

In [53]:
pdb_parser = PDBParser()
structure = pdb_parser.get_structure("prot", f"pdb_files/pdb2rml.ent")

In [60]:
list(structure.get_models())[1].full_id

('2M97', 1)

In [49]:
"".join(residues_str)

'MLSGGLGIAMGVSGMTCAACAAAIGLGLLAMPGVTAAAVALATGTSAVITAPAGTGTAAIGGLIGLLGTHVVTGLAGPAIGGMTCAACAAAIGLALALIGGVAAAPVAPALGTVTVGTAPLGASVSALLGAVALLGTLLLLLGGGAS'

In [48]:
number_to_sequence["11011_1_1_1"]

'MLSEQKEIAMQVSGMTCAACAARIEKGLKRMPGVTDANVNLATETSNVIYDPAETGTAAIQEKIEKLGYHVVTEKAEFDIEGMTCAACANRIEKRLNKIEGVANAPVNFALETVTVEYNPKEASVSDLKEAVDKLGYKLKLKGEQDS'

In [50]:
our_index_to_pdb["11011_1_1_1"]

{'identifier': '2RML_1',
 'score': 1.0,
 'services': [{'service_type': 'sequence',
   'nodes': [{'node_id': 19706,
     'original_score': 291.0,
     'norm_score': 1.0,
     'match_context': [{'sequence_identity': 1.0,
       'evalue': 5.513e-90,
       'bitscore': 291,
       'alignment_length': 147,
       'mismatches': 0,
       'gaps_opened': 0,
       'query_beg': 1,
       'query_end': 147,
       'subject_beg': 1,
       'subject_end': 147,
       'query_length': 147,
       'subject_length': 147,
       'query_aligned_seq': 'MLSEQKEIAMQVSGMTCAACAARIEKGLKRMPGVTDANVNLATETSNVIYDPAETGTAAIQEKIEKLGYHVVTEKAEFDIEGMTCAACANRIEKRLNKIEGVANAPVNFALETVTVEYNPKEASVSDLKEAVDKLGYKLKLKGEQDS',
       'subject_aligned_seq': 'MLSEQKEIAMQVSGMTCAACAARIEKGLKRMPGVTDANVNLATETSNVIYDPAETGTAAIQEKIEKLGYHVVTEKAEFDIEGMTCAACANRIEKRLNKIEGVANAPVNFALETVTVEYNPKEASVSDLKEAVDKLGYKLKLKGEQDS'}]}]}]}